In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat"
outPath = "Results"
foldName = "folds.pickle"

epochs = 100
batch_size = 64
shuffle = True
seed = None

input_data_folder = "Data"
training_data_file = "Training-datasets-PredNTS.txt"
independent_data_file = "independent dataset-PredNTS.txt"

In [2]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, matthews_corrcoef

import math

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def one_hot_encode_nt(sequence, char_dict):
    
    seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
    i = 0
    for single_character in sequence:
        if(single_character.upper() in char_dict.keys()):
            seq_encoded[i][char_dict[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in NT sequence: '+sequence)
    return seq_encoded

In [5]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList



In [6]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [7]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_CORENup(input_seq_shape = (41, 21),
                 conv_filters_per_layer_1 = 10, kernel_length_1 = 10, conv_strides_1 = 1, ## 1st Convolutional layer parameters
                 max_pool_width_1 = 2, max_pool_stride_1 = 2, ## 1st Maxpool layer parameters
                 lstm_decode_units = 10, ## LSTM layer parameters
                 conv_filters_per_layer_2 = 10,  kernel_length_2 = 5, conv_strides_2 = 1, ## 2nd Convolutional layer parameters
                 max_pool_width_2 = 2, max_pool_stride_2 = 2, ## 2nd Maxpool layer parameters
                 dense_decode_units = 32, ## Dense layer parameters
                 prob = 0.5, learn_rate = 0.0005, 
                 loss = 'binary_crossentropy', metrics = None):
    
    beta = 0.001
    
    ######################################################################################################
    ########  SEQUENCE  ##################################################################################
    ######################################################################################################
    
    input1 = tf.keras.layers.Input(shape=input_seq_shape)

    x1 = tf.keras.layers.Conv1D(conv_filters_per_layer_1, kernel_length_1,
                                strides = conv_strides_1, kernel_regularizer = tf.keras.regularizers.l2(beta), 
                                padding = "same")(input1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.MaxPool1D(pool_size = max_pool_width_1, strides = max_pool_stride_1)(x1)
    x1 = tf.keras.layers.Dropout(prob)(x1)

    ## LSTM Path

    x2 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(x1)
    
    x2 = tf.keras.layers.Dropout(prob)(x2)
    
    x2 = tf.keras.layers.Flatten()(x2)

    ## Conv Path

    x3 = tf.keras.layers.Conv1D(conv_filters_per_layer_2, kernel_length_2, strides = conv_strides_2, 
                                kernel_regularizer = tf.keras.regularizers.l2(beta), padding = 'same')(x1)
    x3 = tf.keras.layers.Activation('relu')(x3)
    x3 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width_2, strides = max_pool_stride_2)(x3)
    x3 = tf.keras.layers.Dropout(prob)(x3)
    
    x3 = tf.keras.layers.Flatten()(x3)
    
    x4 = tf.keras.layers.Concatenate(1)([x2,x3])
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(x4)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss)

    return model

In [8]:
DLNN_CORENup().summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 41, 21)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 41, 10)       2110        ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 41, 10)       0           ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 20, 10)       0           ['activation[0][0]']             
                                                                                              

C:\ProgramData\Anaconda3\envs\tf_2_8_py_3_10\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Training

In [9]:
##################################################################################
##### read training file
##################################################################################
train_file_path = os.path.join(input_data_folder, training_data_file)
train_data = pd.read_csv(train_file_path, sep='\t', header=None)
train_data.columns = ['Sequence', 'name', 'id', 'flag', 'label_original', 'type']
train_data.head()

##################################################################################
##### Create dictionary of all characters in the NT sequence 
##################################################################################
all_char_set = set({})
for val in [set(val) for val in train_data['Sequence']]:
    all_char_set = all_char_set.union(val)
all_char_list = list(all_char_set)
all_char_list.sort()
all_char_dict = {}
for i in range(len(all_char_list)):
    all_char_dict[all_char_list[i]] = i
    
##################################################################################
##### Create OHE of sequence
##################################################################################
train_data['OHE_Sequence'] = pd.Series([one_hot_encode_nt(val, all_char_dict) 
                                        for val in train_data["Sequence"]])

##################################################################################
##### Fix the labels
##################################################################################
train_data['label'] = pd.Series([1 if val == 1 else 0 
                                 for val in train_data["label_original"]])

##################################################################################
##### Imbalance the training data
##################################################################################
imbalance_neg_multiplier = 1

for i in range(imbalance_neg_multiplier):
    train_data = pd.concat((train_data, train_data[train_data['label'] == 0]), ignore_index=True)
    
imbalance_pos_multiplier = 0

for i in range(imbalance_pos_multiplier):
    train_data = pd.concat((train_data, train_data[train_data['label'] == 1]), ignore_index=True)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

features = np.array(list(train_data['OHE_Sequence']))
labels = np.array(list(train_data['label']))
labels = labels.reshape((labels.shape[0], 1))

input_seq_shape = features[0].shape

folds = build_kfold(features, labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

In [10]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = fold["X_train"][index_arr], y = fold["y_train"][index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)


Train/Test model on Fold #0.
Epoch 1/100


C:\ProgramData\Anaconda3\envs\tf_2_8_py_3_10\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


45/45 [==============================] - ETA: 0s - loss: 0.7550
Epoch 1: val_loss improved from inf to 0.74003, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold0.hdf5
45/45 [==============================] - 5s 20ms/step - loss: 0.7550 - val_loss: 0.7400
Epoch 2/100
43/45 [===========================>..] - ETA: 0s - loss: 0.7398
Epoch 2: val_loss improved from 0.74003 to 0.72830, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold0.hdf5
45/45 [==============================] - 0s 10ms/step - loss: 0.7429 - val_loss: 0.7283
Epoch 3/100
44/45 [============================>.] - ETA: 0s - loss: 0.7306
Epoch 3: val_loss improved from 0.72830 to 0.72076, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold0.hdf5
45/45 [==============================] - 0s 10ms/step - loss: 0.7316 - val_loss: 0.7208
Epoch 4/100
43/45 [=

45/45 [==============================] - 0s 10ms/step - loss: 0.4689 - val_loss: 0.4677
Epoch 27/100
44/45 [============================>.] - ETA: 0s - loss: 0.4613
Epoch 27: val_loss improved from 0.46771 to 0.46124, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold0.hdf5
45/45 [==============================] - 0s 10ms/step - loss: 0.4622 - val_loss: 0.4612
Epoch 28/100
44/45 [============================>.] - ETA: 0s - loss: 0.4677
Epoch 28: val_loss did not improve from 0.46124
45/45 [==============================] - 0s 9ms/step - loss: 0.4681 - val_loss: 0.4757
Epoch 29/100
43/45 [===========================>..] - ETA: 0s - loss: 0.4437
Epoch 29: val_loss improved from 0.46124 to 0.45663, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold0.hdf5
45/45 [==============================] - 0s 10ms/step - loss: 0.4409 - val_loss: 0.4566
Epoch 30/100
43/45 [============

Epoch 55/100
44/45 [============================>.] - ETA: 0s - loss: 0.3557
Epoch 55: val_loss did not improve from 0.40067
45/45 [==============================] - 0s 9ms/step - loss: 0.3558 - val_loss: 0.4560
Epoch 56/100
39/45 [=========================>....] - ETA: 0s - loss: 0.3566
Epoch 56: val_loss did not improve from 0.40067
45/45 [==============================] - 0s 9ms/step - loss: 0.3620 - val_loss: 0.4054
Epoch 57/100
38/45 [========================>.....] - ETA: 0s - loss: 0.3603
Epoch 57: val_loss did not improve from 0.40067
45/45 [==============================] - 0s 8ms/step - loss: 0.3578 - val_loss: 0.4026
Epoch 58/100
44/45 [============================>.] - ETA: 0s - loss: 0.3594
Epoch 58: val_loss improved from 0.40067 to 0.39969, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold0.hdf5
45/45 [==============================] - 0s 10ms/step - loss: 0.3593 - val_loss: 0.3997
Epoch 59/100
44/45 [==========

...

C:\ProgramData\Anaconda3\envs\tf_2_8_py_3_10\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


41/45 [==========================>...] - ETA: 0s - loss: 0.7495
Epoch 1: val_loss improved from inf to 0.72949, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold4.hdf5
45/45 [==============================] - 2s 19ms/step - loss: 0.7477 - val_loss: 0.7295
Epoch 2/100
43/45 [===========================>..] - ETA: 0s - loss: 0.7320
Epoch 2: val_loss improved from 0.72949 to 0.71668, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold4.hdf5
45/45 [==============================] - 0s 10ms/step - loss: 0.7318 - val_loss: 0.7167
Epoch 3/100
43/45 [===========================>..] - ETA: 0s - loss: 0.7166
Epoch 3: val_loss improved from 0.71668 to 0.70297, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold4.hdf5
45/45 [==============================] - 0s 10ms/step - loss: 0.7162 - val_loss: 0.7030
Epoch 4/100
44/45 [=

Epoch 26/100
45/45 [==============================] - ETA: 0s - loss: 0.4556
Epoch 26: val_loss did not improve from 0.49479
45/45 [==============================] - 0s 9ms/step - loss: 0.4556 - val_loss: 0.4952
Epoch 27/100
39/45 [=========================>....] - ETA: 0s - loss: 0.4445
Epoch 27: val_loss improved from 0.49479 to 0.49093, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold4.hdf5
45/45 [==============================] - 0s 10ms/step - loss: 0.4521 - val_loss: 0.4909
Epoch 28/100
44/45 [============================>.] - ETA: 0s - loss: 0.4492
Epoch 28: val_loss did not improve from 0.49093
45/45 [==============================] - 0s 9ms/step - loss: 0.4518 - val_loss: 0.4947
Epoch 29/100
44/45 [============================>.] - ETA: 0s - loss: 0.4502
Epoch 29: val_loss did not improve from 0.49093
45/45 [==============================] - 0s 9ms/step - loss: 0.4490 - val_loss: 0.4949
Epoch 30/100
44/45 [==========

44/45 [============================>.] - ETA: 0s - loss: 0.3590
Epoch 57: val_loss did not improve from 0.43279
45/45 [==============================] - 0s 9ms/step - loss: 0.3576 - val_loss: 0.4423
Epoch 58/100
43/45 [===========================>..] - ETA: 0s - loss: 0.3493
Epoch 58: val_loss did not improve from 0.43279
45/45 [==============================] - 0s 9ms/step - loss: 0.3513 - val_loss: 0.4338
Epoch 59/100
39/45 [=========================>....] - ETA: 0s - loss: 0.3528
Epoch 59: val_loss did not improve from 0.43279
45/45 [==============================] - 0s 9ms/step - loss: 0.3467 - val_loss: 0.4357
Epoch 60/100
38/45 [========================>.....] - ETA: 0s - loss: 0.3566
Epoch 60: val_loss improved from 0.43279 to 0.43183, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\bestModel-fold4.hdf5
45/45 [==============================] - 0s 10ms/step - loss: 0.3560 - val_loss: 0.4318
Epoch 61/100
39/45 [=======================

45/45 [==============================] - 0s 10ms/step - loss: 0.3113 - val_loss: 0.4011
Epoch 91/100
44/45 [============================>.] - ETA: 0s - loss: 0.3083
Epoch 91: val_loss did not improve from 0.40109
45/45 [==============================] - 0s 9ms/step - loss: 0.3079 - val_loss: 0.4178
Epoch 92/100
45/45 [==============================] - ETA: 0s - loss: 0.3077
Epoch 92: val_loss did not improve from 0.40109
45/45 [==============================] - 0s 9ms/step - loss: 0.3077 - val_loss: 0.4071
Epoch 93/100
44/45 [============================>.] - ETA: 0s - loss: 0.3038
Epoch 93: val_loss did not improve from 0.40109
45/45 [==============================] - 0s 9ms/step - loss: 0.3034 - val_loss: 0.4066
Epoch 94/100
40/45 [=========================>....] - ETA: 0s - loss: 0.3020
Epoch 94: val_loss did not improve from 0.40109
45/45 [==============================] - 0s 8ms/step - loss: 0.3002 - val_loss: 0.4285
Epoch 95/100
44/45 [============================>.] - ETA: 0s - 

## k-fold Training evaluation

In [11]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.865663,0.835751,0.908102,0.743061,0.926959,0.691684
Train,0.960538,0.954947,0.988997,0.925275,0.978169,0.910794


In [12]:
evaluations_df[evaluations_df["Train_Test"] == "Test"]

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
1,0,Test,0.871329,0.841860,"[0.0, 0.0041841004184100415, 0.326359832635983...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9934361, 0.9934361, 0.9158461, 0.9149294, 0...",0.916753,0.757322,0.928571,0.705594
3,1,Test,0.868531,0.846154,"[0.0, 0.004201680672268907, 0.3319327731092437...","[0.0, 0.0, 0.0, 0.0041928721174004195, 0.00419...","[1.9998095, 0.99980944, 0.9540355, 0.9530237, ...",0.894738,0.739496,0.932914,0.697664
5,2,Test,0.843357,0.791667,"[0.0, 0.004201680672268907, 0.1302521008403361...","[0.0, 0.0, 0.0, 0.0020964360587002098, 0.00209...","[1.9950671, 0.9950671, 0.96517354, 0.964324, 0...",0.903969,0.718487,0.905660,0.640557
7,3,Test,0.890756,0.873832,"[0.0, 0.004201680672268907, 0.1680672268907563...","[0.0, 0.0, 0.0, 0.0021008403361344537, 0.00210...","[1.9945223, 0.99452233, 0.9530267, 0.9518967, ...",0.926166,0.785714,0.943277,0.750106
9,4,Test,0.854342,0.825243,"[0.0, 0.004201680672268907, 0.0672268907563025...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9951706, 0.99517053, 0.97784674, 0.97632354...",0.898886,0.714286,0.924370,0.664497


# Independent data

In [13]:
train_features = features
train_labels = labels

In [14]:
##################################################################################
##### read independent data file
##################################################################################
indpe_file_path = os.path.join(input_data_folder, independent_data_file)
indpe_data = pd.read_csv(indpe_file_path, sep='\t', header=None)
indpe_data.columns = ['Sequence', 'name', 'id', 'flag', 'label_original', 'type']
indpe_data.head()
    
##################################################################################
##### Create OHE of sequence
##################################################################################
indpe_data['OHE_Sequence'] = pd.Series([one_hot_encode_nt(val, all_char_dict) 
                                        for val in indpe_data["Sequence"]])

##################################################################################
##### Fix the labels
##################################################################################
indpe_data['label'] = pd.Series([1 if val == 1 else 0 
                                 for val in indpe_data["label_original"]])

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(list(indpe_data['OHE_Sequence']))
indpe_labels = np.array(list(indpe_data['label']))
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

input_seq_shape = indpe_features[0].shape

## Using k-fold Models

### Performance of each k-fold model

In [16]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)
    mcc = matthews_corrcoef(indpe_labels, label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.674776,0.269409,0.677474,0.560591,0.697456,0.202037


In [17]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.654694,0.247706,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.0019569471624266144, 0.00195...","[1.996431, 0.996431, 0.99399334, 0.99226743, 0...",0.666543,0.532020,0.679061,0.163923
1,1,Independent,0.661224,0.259091,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.0009784735812133072, 0.0009784735...","[1.9995158, 0.9995158, 0.9988739, 0.99857545, ...",0.656691,0.561576,0.681018,0.188015
2,2,Independent,0.673469,0.279642,"[0.0, 0.0, 0.0049261083743842365, 0.0049261083...","[0.0, 0.0009784735812133072, 0.000978473581213...","[1.9962618, 0.9962618, 0.9943558, 0.9838357, 0...",0.700785,0.615764,0.684932,0.232250
3,3,Independent,0.695510,0.285354,"[0.0, 0.0, 0.009852216748768473, 0.00985221674...","[0.0, 0.0009784735812133072, 0.000978473581213...","[1.994669, 0.994669, 0.99293256, 0.9872364, 0....",0.684888,0.556650,0.723092,0.222386
4,4,Independent,0.688980,0.275253,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.0009784735812133072, 0.0009784735...","[1.9950845, 0.99508446, 0.99222493, 0.99196047...",0.678463,0.536946,0.719178,0.203610


### Mean score with k-fold models

In [18]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.687347,0.285714,0.689154,0.591133,0.706458,0.233115


### Voting score with k-fold models

In [19]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.691429,0.287105,0.668348,0.581281,0.713307,0.231983


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [20]:
model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

model = tf.keras.models.load_model(current_model_path)

Epoch 1/100


C:\ProgramData\Anaconda3\envs\tf_2_8_py_3_10\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


56/56 [==============================] - ETA: 0s - loss: 0.7537
Epoch 1: val_loss improved from inf to 0.65805, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\_fullModel.hdf5
56/56 [==============================] - 3s 20ms/step - loss: 0.7537 - val_loss: 0.6581
Epoch 2/100
55/56 [============================>.] - ETA: 0s - loss: 0.7303
Epoch 2: val_loss improved from 0.65805 to 0.63543, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\_fullModel.hdf5
56/56 [==============================] - 1s 9ms/step - loss: 0.7304 - val_loss: 0.6354
Epoch 3/100
55/56 [============================>.] - ETA: 0s - loss: 0.7169
Epoch 3: val_loss improved from 0.63543 to 0.61557, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainStrat\5fold\models\_fullModel.hdf5
56/56 [==============================] - 1s 9ms/step - loss: 0.7171 - val_loss: 0.6156
Epoch 4/100
54/56 [==================

56/56 [==============================] - 0s 9ms/step - loss: 0.4166 - val_loss: 0.6556
Epoch 37/100
53/56 [===========================>..] - ETA: 0s - loss: 0.4115
Epoch 37: val_loss did not improve from 0.56775
56/56 [==============================] - 0s 9ms/step - loss: 0.4063 - val_loss: 0.6542
Epoch 38/100
54/56 [===========================>..] - ETA: 0s - loss: 0.3909
Epoch 38: val_loss did not improve from 0.56775
56/56 [==============================] - 0s 9ms/step - loss: 0.3936 - val_loss: 0.6864
Epoch 39/100
52/56 [==========================>...] - ETA: 0s - loss: 0.4043
Epoch 39: val_loss did not improve from 0.56775
56/56 [==============================] - 0s 9ms/step - loss: 0.4052 - val_loss: 0.6279
Epoch 40/100
52/56 [==========================>...] - ETA: 0s - loss: 0.3933
Epoch 40: val_loss did not improve from 0.56775
56/56 [==============================] - 0s 9ms/step - loss: 0.3963 - val_loss: 0.7277
Epoch 41/100
50/56 [=========================>....] - ETA: 0s - l

53/56 [===========================>..] - ETA: 0s - loss: 0.3426
Epoch 75: val_loss did not improve from 0.56775
56/56 [==============================] - 0s 9ms/step - loss: 0.3404 - val_loss: 0.6556
Epoch 76/100
53/56 [===========================>..] - ETA: 0s - loss: 0.3394
Epoch 76: val_loss did not improve from 0.56775
56/56 [==============================] - 0s 9ms/step - loss: 0.3406 - val_loss: 0.6799
Epoch 77/100
50/56 [=========================>....] - ETA: 0s - loss: 0.3311
Epoch 77: val_loss did not improve from 0.56775
56/56 [==============================] - 1s 9ms/step - loss: 0.3304 - val_loss: 0.7319
Epoch 78/100
51/56 [==========================>...] - ETA: 0s - loss: 0.3341
Epoch 78: val_loss did not improve from 0.56775
56/56 [==============================] - 0s 9ms/step - loss: 0.3338 - val_loss: 0.6279
Epoch 79/100
50/56 [=========================>....] - ETA: 0s - loss: 0.3254
Epoch 79: val_loss did not improve from 0.56775
56/56 [==============================] -

In [21]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.794286,0.262136,0.626319,0.133005,0.925636,0.07857


In [22]:
# new
# Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Independent	0.746939	0.298113	0.68632	0.870833	0.298113	0.187083

In [23]:
# 	Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Independent	0.810612	0.344086	0.686108	0.84894	0.344086	0.137501

In [24]:
# out_fasta_file_name = '.'.join((training_data_file.split('.')[0], 'fasta'))
# out_fasta_file_path = os.path.join(input_data_folder, out_fasta_file_name)

# count = 0
# list_seqs = list(train_data['Sequence'])

# with open(out_fasta_file_path, "w") as out_file_obj:
#     for strLine in list_seqs:
        
#         #Output the header
#         out_file_obj.write(">" + str(count+1) + "\n")
#         out_file_obj.write(strLine + "\n")
        
#         count += 1